<a href="https://colab.research.google.com/github/Gabrielghj/BOT-AVANZADO-DE-TRADING/blob/main/BOT_TRADING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar librerías necesarias

In [ ]:
# Instalar librerías necesarias
!pip install backtrader yfinance


In [ ]:
# Importar librerías
import backtrader as bt
import yfinance as yf
import pandas as pd


In [ ]:
# Descargar datos históricos de Bitcoin
ticker = 'BTC-USD'
data = yf.download(ticker, start='2023-01-01', end='2024-01-01')

# Mostrar las primeras filas para verificar
data.head()


In [ ]:
class AdvancedRSIStrategy(bt.Strategy):
    params = (
        ('rsi_period', 14),
        ('sma_fast', 10),
        ('sma_slow', 30),
        ('stop_loss', 0.02),   # 2% Stop Loss
        ('take_profit', 0.04)  # 4% Take Profit
    )

    def __init__(self):
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=self.params.rsi_period)
        self.sma_fast = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.sma_fast)
        self.sma_slow = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.sma_slow)
        self.order = None
        self.buy_price = None

    def next(self):
        # No abrir nuevas órdenes si ya hay una pendiente
        if self.order:
            return

        # Señal de compra
        if not self.position:
            if self.rsi < 30 and self.sma_fast > self.sma_slow:
                self.order = self.buy()
                self.buy_price = self.data.close[0]
                print(f"🔵 COMPRADO a {self.buy_price:.2f} en {self.datetime.date(0)}")

        # Señal de venta (Take Profit o Stop Loss)
        else:
            price_change = (self.data.close[0] - self.buy_price) / self.buy_price

            if price_change >= self.params.take_profit:
                self.order = self.sell()
                print(f"🟢 TAKE PROFIT VENDIDO a {self.data.close[0]:.2f} en {self.datetime.date(0)}")

            elif price_change <= -self.params.stop_loss:
                self.order = self.sell()
                print(f"🔴 STOP LOSS VENDIDO a {self.data.close[0]:.2f} en {self.datetime.date(0)}")


In [ ]:
# Configurar cerebro y cargar datos

# Crear cerebro
cerebro = bt.Cerebro()

# Cargar datos en formato Backtrader
data_feed = bt.feeds.PandasData(dataname=data)
cerebro.adddata(data_feed)

# Agregar estrategia
cerebro.addstrategy(AdvancedRSIStrategy)

# Configuraciones generales
cerebro.broker.setcash(10000.0)
cerebro.broker.setcommission(commission=0.001)
cerebro.addsizer(bt.sizers.FixedSize, stake=1)


In [ ]:
# Ejecutar la simulación

# Mostrar valor inicial
print('🚀 Valor inicial del portafolio: %.2f' % cerebro.broker.getvalue())

# Ejecutar
cerebro.run()

# Mostrar valor final
print('💰 Valor final del portafolio: %.2f' % cerebro.broker.getvalue())


In [ ]:
# Graficar la estrategia
cerebro.plot()
